# Adding New History Fields 

A common source code modification that you may want to do is to output a **new variable**. 

As an example, CAM has a history field that corresponds to the temperature at 500hPa and a number of other pressure levels, but not at 750hPa. Suppose you wanted to output the temperature at 750hPa. The following two
calls are required to add an output variable:
```
call addfld(’T750’,...) (Add a field to the master field list)
call outfld(’T750’,...) (Collect values for this field and write to history file)
```

Each of these are now described in more detail:

## addfld
The sub-routine addfld adds a field to the master list with the following syntax:
```
addfld(fname,type,avgflag,units,long name)
```
where

- **``fname``** =field name
- **``type``** =the type of field. The entry for a single level field would be “horiz only” and the entry
for a 3D field would be “(/ ’lev’ /)”.
- **``avgflag``** =Averaging flag, A = average, I=instantaneous
- **``units``** =the units of the field
- **``long name``** =Field full name

Appropriate values of these parameters for the output of T750 are:

```
call addfld(’T750’,horiz only, ’A’, ’K’, ’Temperature at 750hPa pressure
surface’)
```

with
- **``fname``** = Field name
- **``field``** = array containing field values
- **``idim``** = longitude dimension of field array
- **``c``** = chunk (physics) or latitude (dynamics) index.


## outfld
The subroutine outfld accumulates (or takes the minimum or, maximum of etc as appropriate) the field into the history buffer for the appropriate history tape with the following syntax
```
addfld(fname,type,avgflag,units,long name)
```
where
- **``fname``** = Field name
- **``field``** = array containing field values
- **``idim``** = longitude dimension of field array
- **``c ``**= chunk (physics) or latitude (dynamics) index.


For example:

```
call outfld(’T750’,t750, pcols, lchnk)
```


<div class="alert alert-info">
<strong>Exercise: Add an additional output variable</strong><br><br>
    
Create a case called “b1850_T750” using the compset  B1850  at f19_g17 resolution. 
- Add an output field for the temperature at 750 mbar. 
- Output daily values of T750 and T500 in the “h1” history file. 
- Set the namelist to output a single h1 for the run. 
- Make a 1-month run. 

</div>

<div class="alert alert-warning">  
<details> 
   <summary>
      <font face="Times New Roman" color='blue'>Click here for hints</font>
    </summary>
    
**Tip to add <font face="Courier"><strong>T750</strong></font>**  <br>
- Use <font face="Courier"><strong>T500</strong></font> as a template for your changes.

- Find the subroutine containing <font face="Courier"><strong>T750</strong></font>. 
For instance, in the CESMROOT, use the command: 

```
    grep –r T500 *    
```
    
    
**Tip to check your solution <font face="Courier"><strong>T750</strong></font>** <br>
- When the run is completed, go to your archive directory:
- check the fields <font face="Courier"><strong>T750</strong></font> and <font face="Courier"><strong>T500</strong></font> are in the file <font face="Courier"><strong>h1</strong></font>
- create a file with the difference between <font face="Courier"><strong>T750-T500 </strong></font>
- For instance, you can use ``ncap2`` 
```
    ncap2 -s ’T750_minus_T500=T750-T500' b1850_T750.cam.h1.0001-01-01-00000.nc  T750-T500.nc
```
- Look at the difference with ``ncview``. 

</details>
</div>


<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
            
#Create a new case <font face="Courier"><strong>b1850_T750</strong></font> with the command:
<font face="Courier" color='black'>    
    
    ./create_newcase --case ~/cases/b1850_T750 --compset B1850 --res f19_g17
</font>

#Case <font face="Courier"><strong>setup</strong></font>:
<font face="Courier" color='black'>    
    
    cd ~/cases/b1850_T750
    ./case.setup 
</font>
    
#Locate the file where <font face="Courier"><strong>T500</strong> is computed and copy it into <strong>SourceMods/src.atm</strong></font>:
<font face="Courier" color='black'>  
    
    cp /glade/p/cesm/tutorial/cesm2.1_tutorial2022/components/cam/src/physics/cam/cam_diagnostics.F90 SourceMods/src.cam
</font>
    
#Edit the file <font face="Courier"><strong>SourceMods/src.cam/cam_diagnostics.F90</strong></font> and add the lines:
<font face="Courier" color='black'>  
    
      !++ add a variable for T750 
      call addfld ('T750',       horiz_only,  'A', 'K','Temperature at 750 mbar pressure surface')  
      !++ add a variable for T750 
      if (hist_fld_active('T750')) then 
         call vertinterp(ncol, pcols, pver, state%pmid, 75000._r8, state%t, p_surf, & 
             extrapolate='T', ps=state%ps, phis=state%phis)
         call outfld('T750    ', p_surf, pcols, lchnk )
      end if
</font>
    
#Edit the file <font face="Courier"><strong>user_nl_cam</strong>  and add the lines:</font>
<font face="Courier" color='black'>    
 
    nhtfrq = 0, -24
    mfilt = 1, 31
    fincl2 = 'T750', 'T500'
</font>
    
#Change the <font face="Courier"><strong>run length </strong>:</font>
<font face="Courier" color='black'>    
    
    ./xmlchange STOP_N=1,STOP_OPTION=nmonths
</font>

#Change <font face="Courier"><strong>job queue</strong></font> and <font face="Courier"><strong>account number</strong> (if needed). For instance:</font>
<font face="Courier" color='black'>  
    
    ./xmlchange JOB_QUEUE=regular,PROJECT=UESM0011
</font>

#Build and submit:
<font face="Courier" color='black'>  
    
    qcmd -- ./case.build
    ./case.submit
</font>
    
#When the run is completed, check the fields <font face="Courier"><strong>T750</strong></font> and <font face="Courier"><strong>T500</strong></font> are in the file <font face="Courier"><strong>h1</strong></font>:
<font face="Courier" color='black'>    
    
    cd /glade/scratch/$user/archive/b1850_T750/atm/hist/ 
    ncdump -h b1850_T750.cam.h1.0001-01-01-00000.nc
    
</font>
    
The file should contain:
<font face="Courier" color='black'>    
    
       float T500(time, lat, lon) ;
               T500:units = "K" ;
                T500:long_name = "Temperature at 500 mbar pressure surface" ;
                T500:cell_methods = "time: mean" ;
        float T750(time, lat, lon) ;
                T750:units = "K" ;
                T750:long_name = "Temperature at 750 mbar pressure surface" ;
                T750:cell_methods = "time: mean" ;
</font>

    
#Create a file with the difference between <font face="Courier"><strong>T750-T500 </strong>:</font>
<font face="Courier" color='black'>    
    
    cd /glade/scratch/$user/archive/b1850_T750/atm/hist/ 
    ncap2 -s ‘T750_minus_T500=T750-T500' b1850_T750.cam.h1.0001-01-01-00000.nc  T750-T500.nc 
</font>

#Look at the difference between <font face="Courier"><strong>T750-T500</strong></font> with <font face="Courier"><strong>ncview</strong></font>: 
<font face="Courier" color='black'>    
    
    cd /glade/scratch/$user/archive/b1850_T750/atm/hist/ 
    ncview T750-T500.nc
</font>
  
The field <font face="Courier"><strong>T750-T500</strong></font> looks like:



*<p style="text-align: center;"> 
    ![ncview T750-T500](../../images/sourcemods/ncview_T750-T500.png)  
    Figure: Overview of the CESM directories and the SourceMods directories. </p>*
    
</details>
</div>

